In [ ]:
#%pip install -U transformers accelerate torch
#!pip install numpy==1.24.4

In [4]:
# Import library
from langchain.document_loaders import PyPDFLoader

# Create a document loader for rag_vs_fine_tuning.pdf
loader = PyPDFLoader('https://greencastlein.api.civicclerk.com/v1/Meetings/GetMeetingFileStream(fileId=90,plainText=false)')

# Load the document
data = loader.load()
# Extract just the page content
all_text = "\n".join([doc.page_content for doc in data])
print(all_text[0:1000])

City of Greencastle 
Common Council 
Regular Session 
Mikayla Johnson - Clerk Treasurer 
 
 
 
April 10, 2025 | 7:00 PM 
City Hall 
One North Locust Street, P.O. Box 607 
Greencastle, Indiana 46135 
 
 
 
 
 
AGENDA 
 
I. Call to Order; Roll Call 
II. Pledge of Allegiance 
III. Public Petitions and Comments 
IV. Special Requests 
  A. Noise Ordinance Waiver Request - DePauw Alumni Reunion Weekend 2025 
  B. Noise Ordinance Waiver Request- Phi Kappa Psi Fraternity Social Event With Guests 
  C. Noise Ordinance Waiver Request- Phi Kappa Psi Fraternity Social Event With Guests 
  D. Special Event Street Closure Request- DePauw University Commencement 
V. Department Reports 
  A. Cemetery - Jason Keeney 
  B. Fire Department - Rob Frank 
  i. Fire Inspections - John Burgess 
  C. Planner - Blaine Rout 
  D. Police Department - Chris Jones 
  E. Department of Public Works - Andrew Rogers 
  i. March 2025 Summary Report 
  F. Water & Wastewater Department - Oscar King Jr. 
  i. Report 
  ii.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)

# Split the text
docs = text_splitter.create_documents([all_text])

# Inspect the first few chunks
for i, d in enumerate(docs[:2]):
    print(f"\n--- Chunk {i} ---\n{d.page_content}")


--- Chunk 0 ---
City of Greencastle 
Common Council 
Regular Session 
Mikayla Johnson - Clerk Treasurer 
 
 
 
April 10, 2025 | 7:00 PM 
City Hall 
One North Locust Street, P.O. Box 607 
Greencastle, Indiana 46135 
 
 
 
 
 
AGENDA 
 
I. Call to Order; Roll Call 
II. Pledge of Allegiance 
III. Public Petitions and Comments 
IV. Special Requests 
  A. Noise Ordinance Waiver Request - DePauw Alumni Reunion Weekend 2025 
  B. Noise Ordinance Waiver Request- Phi Kappa Psi Fraternity Social Event With Guests 
  C. Noise Ordinance Waiver Request- Phi Kappa Psi Fraternity Social Event With Guests 
  D. Special Event Street Closure Request- DePauw University Commencement 
V. Department Reports 
  A. Cemetery - Jason Keeney 
  B. Fire Department - Rob Frank 
  i. Fire Inspections - John Burgess 
  C. Planner - Blaine Rout 
  D. Police Department - Chris Jones 
  E. Department of Public Works - Andrew Rogers 
  i. March 2025 Summary Report 
  F. Water & Wastewater Department - Oscar King Jr. 
 

In [7]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from huggingface_hub import login
from langchain_core.prompts import PromptTemplate
full_text = "\n\n".join(doc.page_content for doc in docs)

# Define the summarization prompt
template = """
You are a helpful assistant that summarizes city council meeting agenda packets for public transparency.

The first page of the document is the official agenda. The remaining pages contain details about each agenda item.

Your task is to read the document and generate a bullet-point summary of the agenda items. Use the following format:

• [Agenda Item Title]: [One- to two-sentence summary of the item in plain language]

Be neutral, concise, and citizen-friendly in your language. Avoid technical jargon unless it's essential.

Document:
{agenda}

Summary:
"""
prompt_template = PromptTemplate.from_template(template)

# Load the Hugging Face model
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 500}  # Adjust depending on how long you expect summaries to be
)

# Run the summarization
formatted_prompt = prompt_template.format(agenda=full_text)
response = llm.invoke(formatted_prompt)
print(response)

Device set to use cpu



You are a helpful assistant that summarizes city council meeting agenda packets for public transparency.

The first page of the document is the official agenda. The remaining pages contain details about each agenda item.

Your task is to read the document and generate a bullet-point summary of the agenda items. Use the following format:

• [Agenda Item Title]: [One- to two-sentence summary of the item in plain language]

Be neutral, concise, and citizen-friendly in your language. Avoid technical jargon unless it's essential.

Document:
City of Greencastle 
Common Council 
Regular Session 
Mikayla Johnson - Clerk Treasurer 
 
 
 
April 10, 2025 | 7:00 PM 
City Hall 
One North Locust Street, P.O. Box 607 
Greencastle, Indiana 46135 
 
 
 
 
 
AGENDA 
 
I. Call to Order; Roll Call 
II. Pledge of Allegiance 
III. Public Petitions and Comments 
IV. Special Requests 
  A. Noise Ordinance Waiver Request - DePauw Alumni Reunion Weekend 2025 
  B. Noise Ordinance Waiver Request- Phi Kappa Psi 